In [118]:
%run data_prep.ipynb

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Sum of missing values in column budget : 37491
Sum of missing values in column genres : 2466
Sum of missing values in column id : 0
Sum of missing values in column original_title : 0
Sum of missing values in column production_companies : 12199
Sum of missing values in column production_countries : 6421
Sum of missing values in column release_date : 0
Sum of missing values in column revenue : 39004
Sum of missing values in column title : 0
Sum of missing values in column cast : 2425
Sum of missing values in column director : 865
Sum of missing values in column keywords : 14818


In [119]:
original_movies_df = movies_df.copy() # Useful to go back to the original datframe 
movies_df.head()

,budget,genres,id,original_title,production_companies,production_countries,release_date,revenue,title,cast,director,keywords
0,30000000,"[Animation, Comedy, Family]",862,Toy Story,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva..."
1,65000000,"[Adventure, Fantasy, Family]",8844,Jumanji,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],"[board game, disappearance, based on children'..."
2,0,"[Romance, Comedy]",15602,Grumpier Old Men,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],"[fishing, best friend, duringcreditsstinger, o..."
3,16000000,"[Comedy, Drama, Romance]",31357,Waiting to Exhale,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],"[based on novel, interracial relationship, sin..."
4,0,[Comedy],11862,Father of the Bride Part II,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],"[baby, midlife crisis, confidence, aging, daug..."


In [120]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost as xgb

In [121]:
# Replaces column col of lists of labels with a binary matrix.
# Returns new dataframe and new columns' names.
def binarize_column(df, col):
    mlb = MultiLabelBinarizer()
    return df.join(pd.DataFrame(mlb.fit_transform(df[col]), columns=mlb.classes_, index=df.index), 
                   rsuffix="_suffix").drop(col, axis=1), list(mlb.classes_)

# Takes pandas Series with lists of labels as values.
# Returns a list of labels for which number of occurances > limit.
# Number of labels for 'keywords': > 0 : 18186, > 10 : 2193, > 20 : 1158
def popular_labels(series, limit):
    counts = {}
    for l in series:
        for v in l:
            counts[v] = counts.get(v, 0)+1
    return [k for k,v in counts.items() if v > limit]

# Takes pandas Series with lists of labels as values.
# Creates a new series with only labels that occur at least 'limit' times.
def limit_labels(series, limit):
    new_series = series.copy()
    labels = popular_labels(series, limit)
    for i, l in series.iteritems():
        new_l = [val for val in l if val in labels]
        new_series[i] = new_l
    return new_series

In [122]:
# Transform categorical features into binary matrixes
movies_df['production_countries'] = limit_labels(movies_df['production_countries'], 20)
movies_df, _ = binarize_column(movies_df, 'production_countries')
movies_df['keywords'] = limit_labels(movies_df['keywords'], 20)
movies_df, _ = binarize_column(movies_df, 'keywords')

movies_df.head()

,budget,genres,id,original_title,production_companies,release_date,revenue,title,cast,director,...,yacht,yakuza,young adult,young boy,young love,youth,zatoichi,zombie,zombie apocalypse,zoo
0,30000000,"[Animation, Comedy, Family]",862,Toy Story,[Pixar Animation Studios],1995-10-30,373554033.0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],...,0,0,0,0,0,0,0,0,0,0
1,65000000,"[Adventure, Fantasy, Family]",8844,Jumanji,"[TriStar Pictures, Teitler Film, Interscope Co...",1995-12-15,262797249.0,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],...,0,0,0,0,0,0,0,0,0,0
2,0,"[Romance, Comedy]",15602,Grumpier Old Men,"[Warner Bros., Lancaster Gate]",1995-12-22,0.0,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],...,0,0,0,0,0,0,0,0,0,0
3,16000000,"[Comedy, Drama, Romance]",31357,Waiting to Exhale,[Twentieth Century Fox Film Corporation],1995-12-22,81452156.0,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],...,0,0,0,0,0,0,0,0,0,0
4,0,[Comedy],11862,Father of the Bride Part II,"[Sandollar Productions, Touchstone Pictures]",1995-02-10,76578911.0,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],...,0,0,0,0,0,0,0,0,0,0


In [123]:
train_df, test_df = train_test_split(movies_df, test_size=0.2)

X_cols = ['budget', 'revenue'] # For now only numeric features
X_train = train_df[X_cols]
X_test = test_df[X_cols]

# We need to transform 'genres' since this is multi-label classification
mlb = MultiLabelBinarizer()
# TODO: Can it happen that a genre is in the train set, but not the test set and would it break something?
y_train = mlb.fit_transform(train_df['genres'])
y_test = mlb.fit_transform(test_df['genres'])

genres = []
for x in test_df['genres']:
    for genre in x:
        if genre not in genres:
            genres.append(genre)

In [124]:
# Random forest, only numeric columns
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
y_pred, y_test
accuracy_score(y_test, y_pred)
classification_report(y_test ,y_pred,target_names=genres, output_dict=True)


/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn

{'Comedy': {'precision': 0.34527687296416937,
  'recall': 0.07670043415340087,
  'f1-score': 0.12551805802249852,
  'support': 1382},
 'Romance': {'precision': 0.3317307692307692,
  'recall': 0.09857142857142857,
  'f1-score': 0.15198237885462554,
  'support': 700},
 'Mystery': {'precision': 0.1774193548387097,
  'recall': 0.02743142144638404,
  'f1-score': 0.04751619870410367,
  'support': 401},
 'Drama': {'precision': 0.3667953667953668,
  'recall': 0.07164404223227752,
  'f1-score': 0.11987381703470029,
  'support': 2652},
 'Thriller': {'precision': 0.16666666666666666,
  'recall': 0.027149321266968326,
  'f1-score': 0.04669260700389105,
  'support': 884},
 'Crime': {'precision': 0.1836734693877551,
  'recall': 0.01061320754716981,
  'f1-score': 0.02006688963210702,
  'support': 848},
 'Action': {'precision': 0.5468431771894093,
  'recall': 0.12933526011560695,
  'f1-score': 0.2091936112193222,
  'support': 4152},
 'Western': {'precision': 0.23076923076923078,
  'recall': 0.05046728

In [125]:
# Logistic regression, only numeric columns
clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)
classification_report(y_test ,y_pred,target_names=genres, output_dict=True )

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn

{'Comedy': {'precision': 0.1425233644859813,
  'recall': 0.04413892908827786,
  'f1-score': 0.06740331491712707,
  'support': 1382},
 'Romance': {'precision': 0.09464285714285714,
  'recall': 0.07571428571428572,
  'f1-score': 0.08412698412698412,
  'support': 700},
 'Mystery': {'precision': 0.058165548098434,
  'recall': 0.06483790523690773,
  'f1-score': 0.061320754716981125,
  'support': 401},
 'Drama': {'precision': 0.3580508474576271,
  'recall': 0.06372549019607843,
  'f1-score': 0.10819462227912932,
  'support': 2652},
 'Thriller': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 884},
 'Crime': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 848},
 'Action': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4152},
 'Western': {'precision': 0.081799591002045,
  'recall': 0.07476635514018691,
  'f1-score': 0.078125,
  'support': 535},
 'Animation': {'precision': 0.06157112526539278,
  'recall': 0.06183368869936034,
  'f1-score': 0.0617

In [126]:
X_cols_to_drop = ['genres', 'id', 'original_title', 'production_companies', 
                  'release_date', 'title', 'cast', 'director']
X_train = train_df.drop(columns=X_cols_to_drop, axis=1)
X_test = test_df.drop(columns=X_cols_to_drop, axis=1)

In [127]:
# Random forest with limited keywords and production_countries
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
accuracy_score(y_test, y_pred)
classification_report(y_test ,y_pred,target_names=genres, output_dict=True )

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Comedy': {'precision': 0.6170940170940171,
  'recall': 0.26121562952243127,
  'f1-score': 0.36705643111337066,
  'support': 1382},
 'Romance': {'precision': 0.5606060606060606,
  'recall': 0.15857142857142856,
  'f1-score': 0.24721603563474384,
  'support': 700},
 'Mystery': {'precision': 0.7605633802816901,
  'recall': 0.26932668329177056,
  'f1-score': 0.39779005524861877,
  'support': 401},
 'Drama': {'precision': 0.5360121304018196,
  'recall': 0.26659125188536953,
  'f1-score': 0.35608159153865526,
  'support': 2652},
 'Thriller': {'precision': 0.5938697318007663,
  'recall': 0.1753393665158371,
  'f1-score': 0.2707423580786026,
  'support': 884},
 'Crime': {'precision': 0.5569620253164557,
  'recall': 0.10377358490566038,
  'f1-score': 0.1749502982107356,
  'support': 848},
 'Action': {'precision': 0.6390154968094804,
  'recall': 0.5065028901734104,
  'f1-score': 0.5650947198710198,
  'support': 4152},
 'Western': {'precision': 0.5205479452054794,
  'recall': 0.1420560747663551

In [128]:
# Logistic regression with limited keywords and production_countries
X_train.head()
clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)
classification_report(y_test ,y_pred,target_names=genres, output_dict=True )

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn

{'Comedy': {'precision': 0.1425233644859813,
  'recall': 0.04413892908827786,
  'f1-score': 0.06740331491712707,
  'support': 1382},
 'Romance': {'precision': 0.09464285714285714,
  'recall': 0.07571428571428572,
  'f1-score': 0.08412698412698412,
  'support': 700},
 'Mystery': {'precision': 0.058165548098434,
  'recall': 0.06483790523690773,
  'f1-score': 0.061320754716981125,
  'support': 401},
 'Drama': {'precision': 0.3580508474576271,
  'recall': 0.06372549019607843,
  'f1-score': 0.10819462227912932,
  'support': 2652},
 'Thriller': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 884},
 'Crime': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 848},
 'Action': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4152},
 'Western': {'precision': 0.081799591002045,
  'recall': 0.07476635514018691,
  'f1-score': 0.078125,
  'support': 535},
 'Animation': {'precision': 0.06157112526539278,
  'recall': 0.06183368869936034,
  'f1-score': 0.0617

In [129]:
# XGBoost with limited keywords and production_countries
xgbc = MultiOutputClassifier(xgb.XGBClassifier())
xgbc.fit(X_train, y_train)
y_pred = xgbc.predict(X_test)
print(accuracy_score(y_test, y_pred))
classification_report(y_test ,y_pred,target_names=genres, output_dict=True)

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:05:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:06:04] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:06:48] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:07:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Comedy': {'precision': 0.7760532150776053,
  'recall': 0.2532561505065123,
  'f1-score': 0.38188761593016907,
  'support': 1382},
 'Romance': {'precision': 0.7272727272727273,
  'recall': 0.17142857142857143,
  'f1-score': 0.2774566473988439,
  'support': 700},
 'Mystery': {'precision': 0.851063829787234,
  'recall': 0.29925187032418954,
  'f1-score': 0.4428044280442805,
  'support': 401},
 'Drama': {'precision': 0.7913043478260869,
  'recall': 0.1715686274509804,
  'f1-score': 0.28199566160520606,
  'support': 2652},
 'Thriller': {'precision': 0.6933333333333334,
  'recall': 0.17647058823529413,
  'f1-score': 0.2813345356176736,
  'support': 884},
 'Crime': {'precision': 0.8333333333333334,
  'recall': 0.09433962264150944,
  'f1-score': 0.16949152542372883,
  'support': 848},
 'Action': {'precision': 0.6764593666539489,
  'recall': 0.4270231213872832,
  'f1-score': 0.5235493872729957,
  'support': 4152},
 'Western': {'precision': 0.7222222222222222,
  'recall': 0.14579439252336449,


In [14]:
import gensim.downloader
import numpy as np

# Use pre-trained model based on Wikipedia 2014 + Gigaword (https://nlp.stanford.edu/projects/glove/)
model = gensim.downloader.load('glove-wiki-gigaword-50')

In [34]:
# Takes a string and returns the average of its words vectors.
def string_to_vector(phrase):
    phrase = phrase.lower()
    phrase = ''.join([c for c in phrase if c.isalnum() or c == ' '])
    vectors = np.array([model[word] for word in phrase.split() if word in model])
    if len(vectors) == 0:
        return np.full(50, np.nan)
    return np.average(vectors, axis=0)

# Takes a list of strings and returns the average of its words vectors.
def string_list_to_vector(phrase_list):
    vector_list = [string_to_vector(phrase) for phrase in phrase_list]
    vector_array = np.array([v for v in vector_list if not np.any(np.isnan(v))]) # skip nans
    if len(vector_array) == 0:
        return np.full(50, np.nan)
    return np.average(vector_array, axis=0)

# Replaces column col of strings / lists of strings with a column of word vectors. 
def string_column_to_vector_column(df, col_name, new_names):
    col = df[col_name]
    if type(col[0]) == str:
        fun = string_to_vector
    else:
        fun = string_list_to_vector
    array = np.array([fun(row) for row in col])
    return df.join(pd.DataFrame(array, columns=new_names, index=df.index), 
                   rsuffix="_suffix").drop(col_name, axis=1)

In [103]:
movies_df = original_movies_df.copy() # Reverse all changes to the dataframe

title_cols = ['title'+str(i) for i in range(50)]
keywords_cols = ['keywords'+str(i) for i in range(50)]

# Drop rows with empty genres. Maybe we should do that in data_prep?
movies_df = movies_df[movies_df['genres'].apply(lambda x : x != [])]

# Columns 'title' and 'keywords' are transformed using Word2Vec
movies_df = string_column_to_vector_column(movies_df, 'title', title_cols)
movies_df = string_column_to_vector_column(movies_df, 'keywords', keywords_cols)

# Columns 'production_companies', 'production_countries', 'cast', 'genres'
# are transformed by limiting the labels and using MultiLabelBinarizer.
movies_df['production_companies'] = limit_labels(movies_df['production_companies'], 20)
movies_df, production_companies_cols = binarize_column(movies_df, 'production_companies')
movies_df['production_countries'] = limit_labels(movies_df['production_countries'], 20)
movies_df, production_countries_cols = binarize_column(movies_df, 'production_countries')
movies_df['cast'] = limit_labels(movies_df['cast'], 50)
movies_df, cast_cols = binarize_column(movies_df, 'cast')

movies_df['genres'] = limit_labels(movies_df['genres'], 50)
movies_df, genres_cols = binarize_column(movies_df, 'genres')

# TODO: Transform the column 'director' - it is not multilabel.
movies_df.head()

,budget,id,original_title,release_date,revenue,director,title0,title1,title2,title3,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,30000000,862,Toy Story,1995-10-30,373554033.0,[John Lasseter],0.158795,-0.067820,-0.065135,0.133620,...,0,0,0,0,0,0,0,0,0,0
1,65000000,8844,Jumanji,1995-12-15,262797249.0,[Joe Johnston],-0.025142,-0.792810,-0.613600,-0.107130,...,0,0,0,0,0,0,0,0,0,0
2,0,15602,Grumpier Old Men,1995-12-22,0.0,[Howard Deutch],-0.612903,0.785603,-0.008300,-0.742430,...,0,0,0,0,1,0,0,0,0,0
3,16000000,31357,Waiting to Exhale,1995-12-22,81452156.0,[Forest Whitaker],0.583170,-0.090578,0.333530,-0.928237,...,0,0,0,0,1,0,0,0,0,0
4,0,11862,Father of the Bride Part II,1995-02-10,76578911.0,[Charles Shyer],0.581018,0.635348,-0.503253,-0.341070,...,0,0,0,0,0,0,0,0,0,0


In [104]:
# Names of the columns that we consider.
# I have only included 'budget', 'revenue', 'title' and 'keywords', as otherwise it takes a long time to compute.
# TODO: Should I include genres here?
cols = ['budget', 'revenue']+title_cols+keywords_cols

# Impute the missing values in 'title' and 'keywords'.
# The missing values in the rest of the columns will not change, as they are not equal to np.nan.
# TODO: How to impute these values as well?
imp = IterativeImputer() 
movies_df = movies_df.drop(cols, axis=1).join(
    pd.DataFrame(imp.fit_transform(movies_df[cols]), columns=imp.feature_names_in_, index=movies_df.index))

In [105]:
# Is it bad that we generate new test and train set? 
# Doing it on the same ones would be problematic since we change the format.
train_df, test_df = train_test_split(movies_df, test_size=0.2)

In [106]:
X_cols = ['budget', 'revenue']+title_cols+keywords_cols+production_companies_cols+production_countries_cols+cast_cols
X_train = train_df[X_cols]
X_test = test_df[X_cols]

y_train = train_df[genres_cols]
y_test = test_df[genres_cols]

In [107]:
# Random forest with: 
# - keywords and titles processed by Word2Vec
# - 'production_companies_cols', 'production_countries_cols', 'cast' limited and processed with MultiLabelBinarizer
# - 'budget' and 'revenue'
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(accuracy_score(y_test, y_pred))
classification_report(y_test ,y_pred,target_names=genres_cols, output_dict=True)

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Action': {'precision': 0.8601398601398601,
  'recall': 0.09241172051089407,
  'f1-score': 0.16689280868385345,
  'support': 1331},
 'Adventure': {'precision': 0.8297872340425532,
  'recall': 0.0533515731874145,
  'f1-score': 0.10025706940874035,
  'support': 731},
 'Animation': {'precision': 0.717948717948718,
  'recall': 0.06682577565632458,
  'f1-score': 0.12227074235807861,
  'support': 419},
 'Comedy': {'precision': 0.7557251908396947,
  'recall': 0.14537444933920704,
  'f1-score': 0.24384236453201968,
  'support': 2724},
 'Crime': {'precision': 0.6415094339622641,
  'recall': 0.03926096997690531,
  'f1-score': 0.07399347116430903,
  'support': 866},
 'Documentary': {'precision': 0.8181818181818182,
  'recall': 0.04363636363636364,
  'f1-score': 0.08285385500575375,
  'support': 825},
 'Drama': {'precision': 0.6525599289730689,
  'recall': 0.5362354085603113,
  'f1-score': 0.5887064477372846,
  'support': 4112},
 'Family': {'precision': 0.8333333333333334,
  'recall': 0.052816901

In [115]:
# XGBoost with: 
# - keywords and titles processed by Word2Vec
# - 'production_companies_cols', 'production_countries_cols', 'cast' limited and processed with MultiLabelBinarizer
# - 'budget' and 'revenue'

# Runs about 13 minutes
xgbc = MultiOutputClassifier(xgb.XGBClassifier())
xgbc.fit(X_train, y_train)
y_pred = xgbc.predict(X_test)
print(accuracy_score(y_test, y_pred))
classification_report(y_test ,y_pred,target_names=genres_cols, output_dict=True)

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:44:00] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:44:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:45:09] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:45:46] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

/home/ania/studia/semestr7/ml/ml_project/ml_project/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'Action': {'precision': 0.6610407876230661,
  'recall': 0.35311795642374155,
  'f1-score': 0.4603330068560235,
  'support': 1331},
 'Adventure': {'precision': 0.6495726495726496,
  'recall': 0.2079343365253078,
  'f1-score': 0.31502590673575126,
  'support': 731},
 'Animation': {'precision': 0.8539325842696629,
  'recall': 0.3627684964200477,
  'f1-score': 0.509212730318258,
  'support': 419},
 'Comedy': {'precision': 0.6492734478203435,
  'recall': 0.3608663729809104,
  'f1-score': 0.463898065125059,
  'support': 2724},
 'Crime': {'precision': 0.595108695652174,
  'recall': 0.2528868360277136,
  'f1-score': 0.3549432739059968,
  'support': 866},
 'Documentary': {'precision': 0.6138211382113821,
  'recall': 0.18303030303030304,
  'f1-score': 0.2819794584500467,
  'support': 825},
 'Drama': {'precision': 0.6535556134410002,
  'recall': 0.6101653696498055,
  'f1-score': 0.6311155829455415,
  'support': 4112},
 'Family': {'precision': 0.7976878612716763,
  'recall': 0.24295774647887325,
